## ExaGO Demo 1

In this demo, we will

1. walk through the [ExaGO Github repostory](https://github.com/pnnl/ExaGO)
2. Run an AC optimal power flow, play with [different options available](https://github.com/pnnl/ExaGO/blob/develop/docs/web/opflow.md#options)
3. Run a security-constrained ACOPF
4. Visualize ACOPF output

### 1. [ExaGO Github repostory](https://github.com/pnnl/ExaGO)

### 2 Run AC optimal power flow (OPFLOW)

In [6]:
EXAGO_DIR='/Users/abhy245/software/ExaGO'

netfile='$EXAGO_DIR/datafiles/case9/case9mod.m'

! opflow -netfile $netfile


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.0, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      114
Number of nonzeros in inequality constraint Jacobian.:       72
Number of nonzeros in Lagrangian Hessian.............:       96

Total number of variables............................:       24
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       16
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality constrai

#### print summary, save output to MATPOWER format file, profile code

In [ ]:
! opflow -netfile $netfile -print_output -save_output case9 -opflow_output_format MATPOWER -log_view

#### Use AC power flow initialization, hold generator bus voltage constant within Q limits

In [ ]:
! opflow -netfile $netfile -opflow_initialization ACPF -opflow_genbusvoltage FIXED_WITHIN_QBOUNDS -save_output case9_fixedgenV -print_output

#### Activate load loss (shedding)

In [ ]:
netfile = '$EXAGO_DIR/datafiles/case9/case9mod_loadloss.m'

! opflow -netfile $netfile -opflow_include_loadloss_variables -print_output -save_output case9_loadloss

#### See full list of [OPFLOW options](https://github.com/pnnl/ExaGO/blob/develop/docs/web/opflow.md#options) 

### 3. Run Security-constrained AC optimal power

Note: We run this in an "embarrasingly parallel" fashion as an example, i.e., each ACOPF is solved separately. With HIOP solver installed, one can solve a proper security (contingency)-constrained ACOPF.

In [ ]:
# Run SCOPFLOW with all contingencies, save output to directory, use ACPF initialization for each subproblem
netfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9mod.m'
ctgcfile='/Users/abhy245/software/ExaGO/datafiles/case9/case9.cont'

! cat $ctgcfile

#### Run on 4 processors, print output, save output to directory called SCOPFLOW, select all contingencies in the list

In [ ]:
! mpiexec -n 4 $EXAGO_DIR/install/bin/scopflow -netfile $netfile -ctgcfile $ctgcfile -scopflow_Nc -1 -scopflow_solver EMPAR -print_output -save_output -scopflow_output_directory case9scopf -opflow_initialization ACPF -opflow_output_format MATPOWER

### 4. Visualize ACOPF output

In [2]:
# ACTIVSg Texas system
#netfile='$EXAGO_DIR/install/bin/ACTIVSg2000/case_ACTIVSg2000.m'
#gicfile='$EXAGO_DIR/install/bin/ACTIVSg2000/ACTIVSg2000_GIC_data.gic'

# ACTIVSg WECC system
netfile='$EXAGO_DIR/install/bin/ACTIVSg10k/case_ACTIVSg10k.m'
gicfile='$EXAGO_DIR/install/bin/ACTIVSg10k/ACTIVSg10k_GIC_data.gic'

#### Run ACOPF and save output to JSON format

In [ ]:
! opflow -netfile $netfile -save_output case10000 -opflow_output_format JSON -gicfile $gicfile -opflow_initialization FROMFILE -opflow_ignore_lineflow_constraints

#### Copy over JSON output file to $EXAGO_DIR/viz/data directory

In [7]:
! cp case10000.json $EXAGO_DIR/viz/data

#### Go to $EXAGO_DIR/viz directory and load the JSON output file

In [8]:
import os
os.chdir(os.path.join(EXAGO_DIR,'viz'))
! python3 geninputfile.py case10000.json

#### Launch visualization engine 

One needs to install the required libraries for the visualization engine first. This can be done with running `npm install` in the $EXAGO_DIR/viz directory. See [](https://github.com/pnnl/ExaGO/viz) for instructions on the visualization engine.

In [9]:
! npm start


> geojson@0.0.0 start
> webpack-dev-server --progress --hot --open

10% building 1/1 modules 0 activeℹ ｢wds｣: Project is running at http://localhost:8080/
ℹ ｢wds｣: webpack output is served from /
ℹ ｢wds｣: Content not from webpack is served from /Users/abhy245/software/ExaGO/viz
10% building 1/4 modules 3 active ...4!/Users/abhy245/software/ExaGO/viz/app.jsℹ ｢wdm｣: wait until bundle finished: /
ℹ ｢wdm｣: Hash: 735e438db1f2e70cd4962m39me.js
Version: webpack 4.47.0
Time: 9587ms
Built at: 11/14/2023 4:57:50 PM
     Asset      Size  Chunks                   Chunk Names
    app.js  44.2 MiB     app  [emitted]        app
app.js.map  20.7 MiB     app  [emitted] [dev]  app
Entrypoint app = app.js app.js.map
[0] multi (webpack)-dev-server/client?http://localhost:8080 (webpack)/hot/dev-server.js ./app.js 52 bytes {app} [built]
[./app.js] 44.1 KiB {app} [built]
[./module_casedata.js] 348 bytes {app} [built]
[./node_modules/@deck.gl/core/dist/esm/index.js] 3.49 KiB {app} [built]
[./node_modules/@de